In [10]:
#

import pandas as pd
import matplotlib as plot
import numpy as np
from IPython.display import Markdown as md
from IPython.core.display_functions import display

In [11]:
#
BASE_DIRECTORY="/Users/ammajohn/PycharmProjects/resilienceHub/data_files/"
RDS_DATA="RDS_2022_07_04_00_00_22.csv"
ECS_DATA="ecs_2022_07_04_00_00_22.csv"
ELB_DATA="elbv2_2022_07_04_00_00_22.csv"
KINESIS_DATA="kinesis_2022_07_04_00_00_22.csv"
ASG_DATA="asg_2022_07_04_00_00_22.csv"


In [12]:
# Data Preparation
pd.options.display.float_format = '{:,.0f}'.format
#TODO put in general format
rds_df = pd.read_csv(BASE_DIRECTORY+RDS_DATA)
ecs_df = pd.read_csv(BASE_DIRECTORY+ECS_DATA)
elbv2_df = pd.read_csv(BASE_DIRECTORY+ELB_DATA)

kinesis_df = pd.read_csv(BASE_DIRECTORY+KINESIS_DATA)
asg_df = pd.read_csv(BASE_DIRECTORY+ASG_DATA)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 500)

# Calculate RDS Summary
rds_ha_df = rds_df.loc[(rds_df['Account Name'] == 'aws-red'), ["Product", "Why Triggered "]]
table = rds_ha_df.groupby(by=["Product"]).agg(count=("Product", "count"))
rds_table=table.sort_values(by=['count'], ascending=False)

# Calculate ASG Summary
asg_ha_df =asg_df.loc[(asg_df['Account Name'] == 'aws-red') & ((asg_df['min Size']+asg_df['max Size']+asg_df['Desired Size'])> 0)]
table = asg_ha_df.groupby(by=["Product"]).agg(count=("Product", "count"))
asg_table=table.sort_values(by=['count'], ascending=False)

# Calculate ELB Summary
elb_ha_df = elbv2_df.loc[(elbv2_df['Account Name'] == 'aws-red'), ["Product", "AZ Count", "Why Triggered "]]
table = elb_ha_df.groupby(by=["Product"]).agg(count=("Product", "count"))
elb_table=table.sort_values(by=['count'], ascending=False)

# Calculate Kinesis Sumary
kin_ha_df =kinesis_df.loc[(kinesis_df['Account Name'] == 'aws-red')]
table = kin_ha_df.groupby(by=["Product"]).agg(count=("Product", "count"))
kin_table=table.sort_values(by=['count'], ascending=False)
#
# # Calculate ECS Sumary
ecs_ha_df =ecs_df.loc[(ecs_df['Account Name'] == 'aws-red') ]
table = ecs_ha_df.groupby(by=["Product"]).agg(count=("Product", "count"))
ecs_table=table.sort_values(by=['count'], ascending=False)

# Merge Tables
summary_ha_df = pd.merge(rds_table, asg_table, on="Product", how="outer")
summary_ha_df.rename(columns={'count_x': 'rds_count', 'count_y':'asg_count'}, inplace=True)
summary_ha_df = pd.merge(summary_ha_df, elb_table, on="Product", how="outer")
summary_ha_df.rename(columns={'count': 'elb_count'}, inplace=True)
summary_ha_df = pd.merge(summary_ha_df, kin_table, on="Product", how="outer")
summary_ha_df.rename(columns={'count': 'kin_count'}, inplace=True)
summary_ha_df = pd.merge(summary_ha_df, ecs_table, on="Product", how="outer")
summary_ha_df.rename(columns={'count': 'ecs_count'}, inplace=True)
summary_ha_df = summary_ha_df.fillna(0)



# Production Related Issues (Red Account)

**NOTES:**
- There are a fair number of resources untagged


In [13]:
#
summary_ha_df.fillna(0, inplace=True)
summary_ha_df['Total'] = summary_ha_df['rds_count'] + summary_ha_df['elb_count'] + summary_ha_df['asg_count']+summary_ha_df['kin_count']+summary_ha_df['ecs_count']
#summary_ha_df = summary_ha_df.groupby(by=['Product']).agg(rds_countx=("rds_count","sum"), 
#                                                          elb_countx=("elb_count","sum"))
# (by=["Product"]).agg(count=("Product", "count"))
sum_ha_df = summary_ha_df.groupby(by=['Product']).sum()
sum_ha_df.sort_values(by="Total", ascending=False, inplace=True)
display(sum_ha_df)

grand_total = summary_ha_df['Total'].sum()
total_products = len(summary_ha_df.index)
display(md(f"In total there are {grand_total} defects or warnings to review."))
display(md(f"In total there are {total_products} depts with Multi-AZ related errors."))


,rds_count,asg_count,elb_count,kin_count,ecs_count,Total
Product,,,,,,
None,9,24,0,6,0,39
StudyManagement,0,17,0,0,0,17
dsctgbatch,0,14,0,0,0,14
subjectdataintegration,0,6,0,4,0,10
batch,0,10,0,0,0,10
dalton,9,0,0,0,0,9
Data Fabric Semantic API,6,0,0,0,0,6
mauth,0,4,0,0,0,4
mccadmin,0,4,0,0,0,4


In total there are 197.0 defects or warnings to review.

In total there are 49 depts with Multi-AZ related errors.

# Appendix

## ASG Defects


**NOTES:**
- There are many ASG with 0 min, max, and desired sizes.  There is no cost to have these in this conditon and will not affect Medidata resiliency.  AWS does recommend removing those ASG which do nothing in order to remove unnessary components which could result in confusion in the future.
- Setting Desired Size=0 means the normal instance count is 0 and on failure, service unavailable until an additional instance has ramped up.  For sensitive applications make certain to always have at least 1 running instance and a max size larger than the desired size.  Use Desired Size=0 only for utlity and other non-critical tasks.
- Desired Size=1 means that there is not an instance to fall back to.  DO NOT USE this option for critical tasks.

**QUESTIONS:**
- Are any of these ASGs inactive?
- Are any of these ASGs required for important production applications?
- Are there any issues to removing the ASGs with min,max,and desired equal to 0?

In [14]:
#
for index,row in summary_ha_df.iterrows():
    k = index
    if (k != "None") and (row['asg_count'] > 0):
        hdr = f"### {k} ASG Defects Details"
        df =asg_df.loc[(asg_df['Account Name'] == 'aws-red') & (asg_df['Product'] == k) ]
        df = df.drop(['Default Cooldown', 'Account Number', 'Active Instances','Account Name', 'Propagate Product Tag At Launch'], axis=1)
        if len(df.index) > 0:
            display(md(hdr))
            display(df)

display(md("### ASG Defects by Reason"))
red_asg_df =asg_df.loc[(asg_df['Account Name'] == 'aws-red')]
table = red_asg_df.groupby(by=["Why Triggered "]).agg(count=("Product", "count"))
table=table.sort_values(by=['count'], ascending=False)
table


### StudyManagement ASG Defects Details

,Region,ASG Name,min Size,max Size,Desired Size,AZ Count,Health Check Type,Product,Why Triggered
950,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
951,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
952,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,1,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
953,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,1,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
954,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,1,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
955,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
956,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
957,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,1,3,EC2,StudyManagement,MinSize should be 2 or greater.
958,us-east-1,study-management-queue-innovate-3-ComputeEnvir...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.
959,us-east-1,study-management-queue-production-3-ComputeEnv...,0,2,0,3,EC2,StudyManagement,MinSize should be 2 or greater.


### dsctgbatch ASG Defects Details

,Region,ASG Name,min Size,max Size,Desired Size,AZ Count,Health Check Type,Product,Why Triggered
721,us-east-1,OmicsBatchEc2ComputeEnv-169a8c7d8ec0c19-asg-45...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
722,us-east-1,OmicsBatchEc2ComputeEnv-169a8c7d8ec0c19-asg-e0...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
723,us-east-1,OmicsBatchEc2ComputeEnv-2e6b086fe1a8276-asg-6e...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
724,us-east-1,OmicsBatchEc2ComputeEnv-2e6b086fe1a8276-asg-b8...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
725,us-east-1,OmicsBatchEc2ComputeEnv-2e6b086fe1a8276-asg-bb...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
726,us-east-1,OmicsBatchEc2ComputeEnv-69c36eaaa8cbea0-asg-8c...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
928,us-east-1,ds-ctg-batch-env-ondemand-prod-innovate-asg-1a...,0,2,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
929,us-east-1,ds-ctg-batch-env-ondemand-prod-innovate-asg-2d...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
930,us-east-1,ds-ctg-batch-env-ondemand-prod-innovate-asg-3f...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.
931,us-east-1,ds-ctg-batch-env-ondemand-prod-innovate-asg-e5...,0,1,0,3,EC2,dsctgbatch,MinSize should be 2 or greater.


### batch ASG Defects Details

...

,Region,ASG Name,min Size,max Size,Desired Size,AZ Count,Health Check Type,Product,Why Triggered
771,us-east-1,corral-bakery-production-app,0,1,0,3,EC2,bakery,MinSize should be 2 or greater.


### imedidata ASG Defects Details

,Region,ASG Name,min Size,max Size,Desired Size,AZ Count,Health Check Type,Product,Why Triggered
712,eu-central-1,corral-imedidata-innovate_eucentral1-app,0,0,0,2,ELB,imedidata,MinSize should be 2 or greater. MaxSize should...
713,eu-central-1,corral-imedidata-innovate_eucentral1-background,0,0,0,2,EC2,imedidata,MinSize should be 2 or greater. MaxSize should...
714,eu-central-1,corral-imedidata-innovate_eucentral1-cron,0,0,0,2,EC2,imedidata,MinSize should be 2 or greater. MaxSize should...
715,eu-central-1,corral-imedidata-production_eucentral1-App,0,0,0,2,ELB,imedidata,MinSize should be 2 or greater. MaxSize should...
716,eu-central-1,corral-imedidata-production_eucentral1-Background,0,0,0,2,ELB,imedidata,MinSize should be 2 or greater. MaxSize should...
717,eu-central-1,corral-imedidata-production_eucentral1-cron,0,0,0,2,EC2,imedidata,MinSize should be 2 or greater. MaxSize should...
815,us-east-1,corral-imedidata-bolt-app,0,0,0,3,ELB,imedidata,MinSize should be 2 or greater. MaxSize should...
816,us-east-1,corral-imedidata-bolt-background,0,0,0,3,EC2,imedidata,MinSize should be 2 or greater. MaxSize should...
817,us-east-1,corral-imedidata-bolt-cron,0,0,0,3,EC2,imedidata,MinSize should be 2 or greater. MaxSize should...
818,us-east-1,corral-imedidata-bolt-notification,0,0,0,3,ELB,imedidata,MinSize should be 2 or greater. MaxSize should...


### ASG Defects by Reason

,count
Why Triggered,
MinSize should be 2 or greater. MaxSize should not be the same as the minSize. maxSize cannot be equal to or less than the desired capacity size.,157
MinSize should be 2 or greater.,116
MinSize should be 2 or greater. maxSize cannot be equal to or less than the desired capacity size.,3
maxSize cannot be equal to or less than the desired capacity size.,3


## RDS Defects

### RDS Defects Summary by Product and Engine

In [15]:
# Display RDS Defect Details
for index,row in summary_ha_df.iterrows():
    k = index
    if (k != "None") and (row['rds_count'] > 0):
        hdr = f"### {k} RDS Defects Details"
        df =rds_df.loc[(rds_df['Account Name'] == 'aws-red') & (rds_df['Product'] == k) ]
        df = df.drop(['Account Number', 'Subnet Count','Product', 'Account Name'], axis=1)
        display(md(hdr))
        display(df)



### dalton RDS Defects Details

,Region,DB Instance Identifier,Read Replica,DB Name,Engine,Multi-AZ,Why Triggered
1148,us-east-1,dalton-graph-innovate,False,NaN,neptune,False,RDS does not have multiAZ.
1149,us-east-1,dalton-graph-innovate3,False,NaN,neptune,False,RDS does not have multiAZ.
1150,us-east-1,dalton-graph-innovate-us-east-1c,False,NaN,neptune,False,RDS does not have multiAZ.
1151,us-east-1,dalton-graph-production,False,NaN,neptune,False,RDS does not have multiAZ.
1152,us-east-1,dalton-graph-production2,False,NaN,neptune,False,RDS does not have multiAZ.
1153,us-east-1,dalton-graph-production3,False,NaN,neptune,False,RDS does not have multiAZ.
1154,us-east-1,dalton-graph-production4,False,NaN,neptune,False,RDS does not have multiAZ.
1155,us-east-1,dalton-graph-production5,False,NaN,neptune,False,RDS does not have multiAZ.
1156,us-east-1,dalton-graph-production-us-east-1c,False,NaN,neptune,False,RDS does not have multiAZ.


### Data Fabric Semantic API RDS Defects Details

,Region,DB Instance Identifier,Read Replica,DB Name,Engine,Multi-AZ,Why Triggered
1157,us-east-1,data-fabric-semantic-api-innovate-instance-0,False,NaN,neptune,False,RDS does not have multiAZ.
1158,us-east-1,data-fabric-semantic-api-innovate-instance-1,False,NaN,neptune,False,RDS does not have multiAZ.
1159,us-east-1,data-fabric-semantic-api-innovate-instance-2,False,NaN,neptune,False,RDS does not have multiAZ.
1160,us-east-1,data-fabric-semantic-api-production-instance-0,False,NaN,neptune,False,RDS does not have multiAZ.
1161,us-east-1,data-fabric-semantic-api-production-instance-1,False,NaN,neptune,False,RDS does not have multiAZ.
1162,us-east-1,data-fabric-semantic-api-production-instance-2,False,NaN,neptune,False,RDS does not have multiAZ.


### coder RDS Defects Details

...

,Region,DB Instance Identifier,Read Replica,DB Name,Engine,Multi-AZ,Why Triggered
1181,us-east-1,mdd9c7dec6f3055,False,NaN,sqlserver-web,False,RDS does not have multiAZ.


### harpy RDS Defects Details

,Region,DB Instance Identifier,Read Replica,DB Name,Engine,Multi-AZ,Why Triggered
1182,us-east-1,mdda02ae620058b,False,NaN,sqlserver-se,False,RDS does not have multiAZ.


### neobo4 RDS Defects Details

,Region,DB Instance Identifier,Read Replica,DB Name,Engine,Multi-AZ,Why Triggered
1188,us-east-1,nro1ie9snxcjq8,False,NaN,sqlserver-ee,False,RDS does not have multiAZ.


# ELB Defects

**QUESTION:**
- What are these Product codes and are they important?




In [16]:
#
for index,row in summary_ha_df.iterrows():
    k = index
    if (k != "None") and (row['elb_count'] > 0):
        hdr = f"### {k} ELB Defects Details"
        df =elbv2_df.loc[(elbv2_df['Account Name'] == 'aws-red') & (elbv2_df['Product'] == k) ]
        df = df.drop(['Account Number', 'Product', 'Account Name'], axis=1)
        display(md(hdr))
        display(df)

### infoblox ELB Defects Details

,Region,Load Balancer Name,DNS Name,Scheme,VPC ID,LB Type,AZ Count,Why Triggered
2,us-east-1,infoblox-production,infoblox-production-d749758dde761f01.elb.us-ea...,internet-facing,vpc-04e10c60,network,1,MultiAZ requires at least 2 Availability Zones.


### security ELB Defects Details

,Region,Load Balancer Name,DNS Name,Scheme,VPC ID,LB Type,AZ Count,Why Triggered
1,us-east-1,insightvm-production,insightvm-production-824935b0621f330b.elb.us-e...,internet-facing,vpc-04e10c60,network,1,MultiAZ requires at least 2 Availability Zones.


# Kinesis Defects

**NOTES:**
-

In [17]:
#
for index,row in summary_ha_df.iterrows():
    k = index
    if (k != "None") and (row['kin_count'] > 0):
        hdr = f"### {k} Kinesis Defects Details"
        df =kinesis_df.loc[(kinesis_df['Account Name'] == 'aws-red') & (kinesis_df['Product'] == k) ]
        df = df.drop(['Account Number', 'Product', 'Account Name'], axis=1)
        display(md(hdr))
        display(df)

### subjectdataintegration Kinesis Defects Details

,Region,Stream Name,Stream Status,Shard Count,Retention Period Hours,Why Triggered
123,us-east-1,subject_data_integration_processed_innovate,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
124,us-east-1,subject_data_integration_processed_production,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
125,us-east-1,subject_data_integration_unprocessed_innovate,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
126,us-east-1,subject_data_integration_unprocessed_production,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...


### milestone Kinesis Defects Details

,Region,Stream Name,Stream Status,Shard Count,Retention Period Hours,Why Triggered
117,us-east-1,milestone_publisher_innovate,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
118,us-east-1,milestone_publisher_output_innovate,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
119,us-east-1,milestone_publisher_output_production,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
120,us-east-1,milestone_publisher_production,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...


### appconnect Kinesis Defects Details

,Region,Stream Name,Stream Status,Shard Count,Retention Period Hours,Why Triggered
110,us-east-1,appconnect_innovate,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
111,us-east-1,appconnect_production,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...


### mingest Kinesis Defects Details

,Region,Stream Name,Stream Status,Shard Count,Retention Period Hours,Why Triggered
121,us-east-1,mingest-innovate-modm,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...
122,us-east-1,mingest-production-modm,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...


### medistrano Kinesis Defects Details

,Region,Stream Name,Stream Status,Shard Count,Retention Period Hours,Why Triggered
114,us-east-1,medistrano_event_stream,ACTIVE,1,24,Kinesis should have > 1 shard...currently 1 sh...


# ECS Defects

**NOTES:**
- There are no ECS Records in this run with errors

In [18]:
ecs_df

,Account Name,Account Number,Region,Cluster Name,Cluster Status,Registered Container Instance Count,Active Service Count,Capacity Providers,Service Name,Service Status,Placement Constraints,Placement Strategy,Desired Count,Enabled ECSManaged Tags,Product,Why Triggered
